In [1]:
import pandas as pd

In [2]:
merged = pd.read_csv('merge.csv')
merged = merged.sort_values(by='application_id')

In [3]:
merged.loc[merged['loan_limit'] != merged['loan_limit'], 'loan_limit'] = merged['loan_limit'].mean()
merged.loc[merged['loan_rate'] != merged['loan_rate'], 'loan_rate'] = merged['loan_rate'].mean()
merged.loc[merged['credit_score'] != merged['credit_score'], 'credit_score'] = merged['credit_score'].mean()
merged.loc[merged['personal_rehabilitation_yn'] != merged['personal_rehabilitation_yn'], 'personal_rehabilitation_yn'] = 0
merged.loc[merged['personal_rehabilitation_complete_yn'] != merged['personal_rehabilitation_complete_yn'], 'personal_rehabilitation_complete_yn'] = 0
merged.loc[merged['working_period'] != merged['working_period'], 'working_period'] = 0
merged.loc[merged['existing_loan_cnt'] != merged['existing_loan_cnt'], 'existing_loan_cnt'] = 0

In [4]:
merged['tmp'] = merged['existing_loan_amt'] / merged['existing_loan_cnt']
tmp_mean = merged['tmp'].mean()

In [5]:
merged.loc[merged['existing_loan_amt'] != merged['existing_loan_amt'], 'existing_loan_amt'] = tmp_mean

In [6]:
merged = merged.drop(['bank_id', 'loanapply_insert_time', 'user_id', 'birth_year', 'gender', 'insert_time', 'tmp'],
                       axis = 1)
training_set = merged.dropna(subset=['is_applied'], axis=0)

In [8]:
x = training_set[training_set.columns.difference(['is_applied', 'application_id', 'product_id'])]
y = training_set.loc[:, training_set.columns == 'is_applied']

for col in x:
    if col == 'income_type' or col == 'employment_type' or col == 'houseown_type' or col == 'purpose':
        x[col] = x[col].astype('str')
    elif col == 'loan_rate':
        x[col] = x[col].astype('float')
    else:
        x[col] = x[col].astype('int')

/var/folders/16/4vbkj73s3qlb5w1xc7nxm92m0000gn/T/ipykernel_39400/2073842526.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x[col] = x[col].astype('int')
/var/folders/16/4vbkj73s3qlb5w1xc7nxm92m0000gn/T/ipykernel_39400/2073842526.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x[col] = x[col].astype('str')
/var/folders/16/4vbkj73s3qlb5w1xc7nxm92m0000gn/T/ipykernel_39400/2073842526.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_in

In [9]:
x_dic = x.to_dict('records')

In [10]:
from sklearn.feature_extraction import DictVectorizer
one_hot_encoder = DictVectorizer()
onehot_X = one_hot_encoder.fit_transform(x_dic)

In [11]:
print(onehot_X.shape)
print(y.shape)

(10270011, 32)
(10270011, 1)


In [12]:
%%time
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV

train_X, test_X = onehot_X[0 : 6000000], onehot_X[6000000:]
train_Y, test_Y = y[0 : 6000000], y[6000000:]

decision_tree = DecisionTreeClassifier(criterion = 'gini', min_samples_split=30)
parameters = {'max_depth': [3, 10, None]}

grid_search = GridSearchCV(decision_tree, parameters, cv=3, scoring='accuracy')

grid_search.fit(train_X, train_Y)
print(grid_search.best_params_)

{'max_depth': 10}
CPU times: user 16min 48s, sys: 9.97 s, total: 16min 58s
Wall time: 22min 44s


In [13]:
from sklearn.metrics import classification_report
model_best = grid_search.best_estimator_
prediction = model_best.predict(test_X)

print(classification_report(test_Y, prediction))

              precision    recall  f1-score   support

         0.0       0.95      1.00      0.97   4036232
         1.0       0.53      0.02      0.04    233779

    accuracy                           0.95   4270011
   macro avg       0.74      0.51      0.50   4270011
weighted avg       0.92      0.95      0.92   4270011



In [14]:
import graphviz
from sklearn.tree import export_graphviz
import pydotplus
import pydot
from six import StringIO
from IPython.core.display import Image

In [15]:
export_graphviz(grid_search.best_estimator_, out_file="tree.dot", class_names = ['not_applied', 'applied'],
                feature_names = one_hot_encoder.get_feature_names_out(),
                 impurity = True, filled=True)

In [16]:
graph = pydotplus.graph_from_dot_file('tree.dot')
graph.write_png('tree.png')

dot: graph is too large for cairo-renderer bitmaps. Scaling by 0.1863 to fit



True

In [27]:
prediction_datas = merged[merged['is_applied'].isnull()]
merged.isnull().sum()

application_id                               0
product_id                                   0
loan_limit                                   0
loan_rate                                    0
is_applied                             3257239
credit_score                                 0
yearly_income                                6
income_type                                  0
working_period                               0
employment_type                              0
houseown_type                                0
desired_amount                               0
purpose                                      0
personal_rehabilitation_yn                   0
personal_rehabilitation_complete_yn          0
existing_loan_cnt                            0
existing_loan_amt                            0
dtype: int64

In [29]:
pred_x = prediction_datas[prediction_datas.columns.difference(['is_applied', 'application_id', 'product_id'])]
pred_x.loc[pred_x['yearly_income'] != pred_x['yearly_income'], 'yearly_income'] = pred_x['yearly_income'].mean()
for col in pred_x:
    if col == 'income_type' or col == 'employment_type' or col == 'houseown_type' or col == 'purpose':
        pred_x[col] = pred_x[col].astype('str')
    elif col == 'loan_rate':
        pred_x[col] = pred_x[col].astype('float')
    else:
        pred_x[col] = pred_x[col].astype('int')

/var/folders/16/4vbkj73s3qlb5w1xc7nxm92m0000gn/T/ipykernel_39400/3597204490.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pred_x[col] = pred_x[col].astype('int')
/var/folders/16/4vbkj73s3qlb5w1xc7nxm92m0000gn/T/ipykernel_39400/3597204490.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pred_x[col] = pred_x[col].astype('str')
/var/folders/16/4vbkj73s3qlb5w1xc7nxm92m0000gn/T/ipykernel_39400/3597204490.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Tr

In [30]:
pred_dic = pred_x.to_dict('records')

In [31]:
one_hot_encoder = DictVectorizer()
onehot_pred = one_hot_encoder.fit_transform(pred_dic)

In [32]:
model_best = grid_search.best_estimator_
preds = model_best.predict(onehot_pred)

In [37]:
prediction_datas = prediction_datas[['application_id', 'product_id', 'is_applied']]
prediction_datas.loc[:, 'is_applied'] = preds
prediction_datas

,application_id,product_id,is_applied
8367639,4,191,0.0
8367638,4,220,0.0
12740391,8,114,0.0
12740390,8,85,0.0
12740392,8,29,0.0
...,...,...,...
11875089,2167778,258,0.0
9041624,2167791,29,0.0
7136127,2167822,149,0.0
7136128,2167822,65,0.0


In [38]:
submission = pd.read_csv('submission.csv')
submission

,application_id,product_id,is_applied
0,4,220,NaN
1,4,191,NaN
2,8,29,NaN
3,8,159,NaN
4,8,85,NaN
...,...,...,...
3255189,2167778,258,NaN
3255190,2167791,29,NaN
3255191,2167822,149,NaN
3255192,2167822,157,NaN


In [41]:
prediction_datas.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3257239 entries, 8367639 to 7136129
Data columns (total 3 columns):
 #   Column          Dtype  
---  ------          -----  
 0   application_id  int64  
 1   product_id      int64  
 2   is_applied      float64
dtypes: float64(1), int64(2)
memory usage: 99.4 MB


In [42]:
result_dic = prediction_datas.to_dict('split')
print(result_dic['data'][0][2])

0.0


In [46]:
%%time
dic = {}
for i in prediction_datas.index:
    key = (prediction_datas.loc[i, 'application_id'], prediction_datas.loc[i, 'product_id'])
    val = prediction_datas.loc[i, 'is_applied']
    dic[key] = val

CPU times: user 1min 1s, sys: 639 ms, total: 1min 2s
Wall time: 1min 2s


In [47]:
print(dic[(4, 191)])

0.0


In [48]:
def fill_applied(submission):
    app_id = submission.application_id
    prod_id = submission.product_id
    
    is_app = dic[(app_id, prod_id)]
    
    return pd.Series([is_app])

In [50]:
%%time
for i in submission.index:
    key = (submission.loc[i, 'application_id'], submission.loc[i, 'product_id'])
    submission.loc[i, 'is_applied'] = dic[key]

submission

CPU times: user 2min 23s, sys: 520 ms, total: 2min 24s
Wall time: 2min 24s


,application_id,product_id,is_applied
0,4,220,0.0
1,4,191,0.0
2,8,29,0.0
3,8,159,0.0
4,8,85,0.0
...,...,...,...
3255189,2167778,258,0.0
3255190,2167791,29,0.0
3255191,2167822,149,0.0
3255192,2167822,157,0.0


In [51]:
submission.to_csv("done.csv", index=False)